<a href="https://colab.research.google.com/github/Alihassan7726/Text-classification-using-ULMFiT/blob/main/Text_Classification_using_ULMFiT_Part_2.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
# import libraries
import fastai
from fastai import *
from fastai.text import * 
import pandas as pd
import numpy as np
from functools import partial
import io
import os
pd.set_option('display.width',1000)
pd.set_option('max_colwidth', 1000) 
import pandas as pd

In [ ]:
#import dataset
from sklearn.datasets import fetch_20newsgroups
dataset = fetch_20newsgroups(shuffle=True, random_state=1, remove=('headers', 'footers', 'quotes'))
# create a dataframe
df = pd.DataFrame({'label':dataset.target,
                   'text':dataset.data})
df.head()

,label,text
0,17,"Well i'm not sure about the story nad it did seem biased. What\nI disagree with is your statement that the U.S. Media is out to\nruin Israels reputation. That is rediculous. The U.S. media is\nthe most pro-israeli media in the world. Having lived in Europe\nI realize that incidences such as the one described in the\nletter have occured. The U.S. media as a whole seem to try to\nignore them. The U.S. is subsidizing Israels existance and the\nEuropeans are not (at least not to the same degree). So I think\nthat might be a reason they report more clearly on the\natrocities.\n\tWhat is a shame is that in Austria, daily reports of\nthe inhuman acts commited by Israeli soldiers and the blessing\nreceived from the Government makes some of the Holocaust guilt\ngo away. After all, look how the Jews are treating other races\nwhen they got power. It is unfortunate.\n"
1,0,"\n\n\n\n\n\n\nYeah, do you expect people to read the FAQ, etc. and actually accept hard\natheism? No, you need a little leap of faith, Jimmy. Your logic runs out\nof steam!\n\n\n\n\n\n\n\nJim,\n\nSorry I can't pity you, Jim. And I'm sorry that you have these feelings of\ndenial about the faith you need to get by. Oh well, just pretend that it will\nall end happily ever after anyway. Maybe if you start a new newsgroup,\nalt.atheist.hard, you won't be bummin' so much?\n\n\n\n\n\n\nBye-Bye, Big Jim. Don't forget your Flintstone's Chewables! :) \n--\nBake Timmons, III"
2,17,"Although I realize that principle is not one of your strongest\npoints, I would still like to know why do do not ask any question\nof this sort about the Arab countries.\n\n If you want to continue this think tank charade of yours, your\nfixation on Israel must stop. You might have to start asking the\nsame sort of questions of Arab countries as well. You realize it\nwould not work, as the Arab countries' treatment of Jews over the\nlast several decades is so bad that your fixation on Israel would\nbegin to look like the biased attack that it is.\n\n Everyone in this group recognizes that your stupid 'Center for\nPolicy Research' is nothing more than a fancy name for some bigot\nwho hates Israel."
3,11,"Notwithstanding all the legitimate fuss about this proposal, how much\nof a change is it? ATT's last product in this area (a) was priced over\n$1000, as I suspect 'clipper' phones will be; (b) came to the customer \nwith the key automatically preregistered with government authorities. Thus,\naside from attempting to further legitimize and solidify the fed's posture,\nClipper seems to be ""more of the same"", rather than a new direction.\n Yes, technology will eventually drive the cost down and thereby promote\nmore widespread use- but at present, the man on the street is not going\nto purchase a $1000 crypto telephone, especially when the guy on the other\nend probably doesn't have one anyway. Am I missing something?\n The real question is what the gov will do in a year or two when air-\ntight voice privacy on a phone line is as close as your nearest pc. That\nhas got to a problematic scenario for them, even if the extent of usage\nnever surpasses the 'underground' stature of ..."
4,10,"Well, I will have to change the scoring on my playoff pool. Unfortunately\nI don't have time right now, but I will certainly post the new scoring\nrules by tomorrow. Does it matter? No, you'll enter anyway!!! Good!\n\n--\n Keith Keller\t\t\t\tLET'S GO RANGERS!!!!!\n\t\t\t\t\t\tLET'S GO QUAKERS!!!!!\n\tkkeller@mail.sas.upenn.edu\t\tIVY LEAGUE CHAMPS!!!!"


In [ ]:
print(df.shape)
dataset.target_names


(11314, 2)


['alt.atheism',
 'comp.graphics',
 'comp.os.ms-windows.misc',
 'comp.sys.ibm.pc.hardware',
 'comp.sys.mac.hardware',
 'comp.windows.x',
 'misc.forsale',
 'rec.autos',
 'rec.motorcycles',
 'rec.sport.baseball',
 'rec.sport.hockey',
 'sci.crypt',
 'sci.electronics',
 'sci.med',
 'sci.space',
 'soc.religion.christian',
 'talk.politics.guns',
 'talk.politics.mideast',
 'talk.politics.misc',
 'talk.religion.misc']

In [ ]:
# We will be taking only 2 classes ( for binary classification )
df = df[df['label'].isin([1,10])]
df = df.reset_index(drop = True)
df['label'].value_counts()

10    600
1     584
Name: label, dtype: int64

In [ ]:
import re
import nltk
from nltk.stem import WordNetLemmatizer as wnl
from nltk.corpus import stopwords
from nltk.tokenize import word_tokenize
nltk.download('stopwords')
stpwrds = set(stopwords.words('english'))
stpwrds.discard('not')
stpwrds.discard('no')


[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Unzipping corpora/stopwords.zip.


In [ ]:
nltk.download('wordnet')

[nltk_data] Downloading package wordnet to /root/nltk_data...
[nltk_data]   Unzipping corpora/wordnet.zip.


True

In [ ]:
def clean_text(text):  
    lower = text.lower()
    words = re.sub(r"\n\t\s\r"," ",lower)
    words = re.sub(r"(@[A-Za-z]+)|([^A-Za-z \t])|(\w+:\/\/\S+)|^rt|http.+?", " ", lower ) 
    #words = re.sub(r"\n\t\s\r"," ") 
    #pattren = r"[^\w\s\d]"
    #words = re.sub(pattren, "", lower)
    words2 = words.split()
    #print(words2)
    final_words =  [wnl().lemmatize(word , pos = 'v') for word in words2 if word not in stopwords.words('english')]
    final_words = ' '.join(final_words)
    return(final_words)
    
df['Cleaned-text'] = df['text'].apply(clean_text)
df.drop(columns = 'text', inplace = True)
df.head()

,label,Cleaned-text
0,10,well change score playoff pool unfortunately time right certainly post new score rule tomorrow matter enter anyway good keith keller let go rangers let go quakers kkeller sas upenn edu ivy league champ
1,1,archive name graphics resources list part last modify computer graphics resource list weekly post part last change april many faqs include list available archive site pit manager mit edu alias rtfm mit edu directory pub usenet news answer name faq archive appear archive name line top article faq archive graphics resources list part mail server machine send e mail message mail server manager mit edu contain keyword help without quote message body see many place list see item place find resource list information items change arrange subject order fir better k article limit plan change headers soon careful resource list key sure remain subject line place find resource list graphics editors public domain free shareware systems plot package molecular visualization stuff think make post bi weekly think line get change character front add line prepended remove line remove use diff locate change text c copyright nikolaos c fotis copy freely file provide keep copyright notice intact compile...
2,10,course mike ramsey one time captain buffalo prior trade pittsburgh currently penguins former captain real captain lemieux play rotate season even c mario even troy loney wear c pen jay
3,10,promise would give name panther president huizenga announce team name announce bill torrey name first president panthers little bio sun sentinel torrey architect four consecutive stanley cup champion persident general manager new york islanders throughout years nhl bill torrey bow tie become much signature andre agassi hair panthers introduce uniform insignia ticket price information early next month meantime huizenga leave day day operation hand torrey bob clarke vp gm florida panthers choose name south florida nhl team focus attention endanger species florida panthers everglades national park big cypress national preserve part southwestern florida panther quickest strike cat torrey say hopefully way play ice bio torrey huizenga first man hire expansion islanders one responsible guide islanders four consecutive stanley cup championships first eight season consecutive win season executive vice president california golden seal torrey watch seal go play off second nhl season guess co...
4,10,game adirondack cdi adirondack lead series first round springfield indians vs providence bruins gm springfield providence gm springfield providence gm providence springfield gm providence springfield gm springfield providence gm providence springfield gm springfield providence cd islanders vs adirondack red wing gm adirondack cdi gm cdi adirondack gm adirondack cdi gm adirondack cdi gm cdi adirondack gm adirondack cdi gm cdi adirondack baltimore skipjacks binghamton rangers gm baltimore binghamton gm baltimore binghamton gm binghamton baltimore gm binghamton baltimore gm baltimore binghamton gm binghmaton baltimore gm baltimore binghamton utica devil vs rochester americans gm utica rochester gm utica rochester gm rochester utica gm rochester utica gm utica rochester gm rochester utica gm utica rochester moncton hawk vs st john maple leaf gm st john moncton gm moncton vs st john halifax gm st john moncton gm st john moncton gm moncton vs st john halifax gm st john moncton gm moncton...


## 1.Data For language Model

In [ ]:
# Batch-size and BPTT
bs,bptt=32,40


### Creating a forward Model

In [ ]:
data_lm = (TextList.from_df(df, cols='Cleaned-text')
                   .split_by_rand_pct(0.1 , seed = 42)
                   .label_for_lm()
                   .databunch(bs=bs, bptt=bptt))

data_lm.save('data_lm.pkl')


/usr/local/lib/python3.7/dist-packages/fastai/core.py:302: VisibleDeprecationWarning: Creating an ndarray from ragged nested sequences (which is a list-or-tuple of lists-or-tuples-or ndarrays with different lengths or shapes) is deprecated. If you meant to do this, you must specify 'dtype=object' when creating the ndarray
  return np.array(a, dtype=dtype, **kwargs)


In [ ]:
del(data_lm)
path = "/content/"
data_lm = load_data(path , 'data_lm.pkl', bs=bs, bptt=bptt)
data_lm.show_batch()


idx,text
0,part last modify computer graphics resource list weekly post part last change april many faqs include list available archive site pit manager mit edu alias rtfm mit edu directory pub usenet news answer name faq archive appear archive name line
1,european sit carry add xxunk available radiance xxunk xxunk se pub sipp sipp tar z sipp scan line z buffer phong shade renderer jonas xxunk jonas xxunk se xxunk fr xxunk vm pray ray tracer spd nff many non spd
2,car radio park lot thankful strong radio miles pittsburgh xxunk usually good believe pick middle day xxbos anyone really believe cap beat let xxunk pen may xxunk one game put definitely lose one game remember regular season mean much come
3,might except might something x windows pcs know use think good email xxunk xxunk gov au dv x common xxunk xxunk corporation xxunk x software use dv x yet read xxunk xxunk send sound pretty great allow multiple do machine
4,g pts ashton berube macinnis musil nieuwendyk otto paslawski ranheim reichel roberts skrudland stern suter yawney san jose power play scorer g pts garpenlov gaudreau kisio odgers wood vancouver los angeles first period los angeles robitaille gretzky sandstrom vancouver babych


### Creating a backward Model


In [ ]:
data_bwd = load_data(path, 'data_lm.pkl', bs=bs, bptt=bptt, backwards=True)
data_bwd.show_batch()


idx,text
0,personally take hey streak unbeaten season regular far unbeaten game year last finals pen meet row blackhawks chicago tie streak playoff streak pen streak win game xxunk want might xxbos xxunk kiss xxunk hint siggraph xxunk xxunk g e surface
1,system software engineer navy xxunk sponsor maryland bethesda center research taylor david formerly center warfare surface naval division carderock june tuesday seminar reality virtual visualization scientific navy presentations call nd xxbos lssu maine xxbos xxunk also dve right xxunk wdve
2,use shit like play canucks xxunk first though big come probert win detroit think yes series playoff intense possibly war seven even maybe six wing though xxunk three two win might toronto playoffs lousy montreal find soon xxunk much leaf
3,basis xxunk state xxunk lease current oilers community business northlands office political hold power wake want pocklington xxbos source c even maybe algorithm understand easy anybody surface two intersections problem xxbos claim com ibm xxunk xxunk rate clock doubt seriously
4,idl lang comp also see available discount quantity educational cost progress version mac windows microsoft sun sgi rs ibm hp ultrix vms dec environments windows ms motif look open applications idl gui build system base widget map xxunk statistics basic


## Fine-tuning the forward language model

The vocabulary of the two datasets ( BBC and Wikipedia ) are slightly different, so when loading the weights, we take care to put the embedding weights at the right place, and we randomly initiliaze the embeddings for words in the BBC vocabulary that weren't in the wikitext-103 vocabulary of our pretrained model. 

This is all done in just 1st line of code below

In [ ]:
learn = language_model_learner(data_lm, AWD_LSTM, metrics=[accuracy, Perplexity()], path=path, wd=0.1).to_fp16(clip=0.1) # # Speeds up training on a GPU


In [ ]:
lr = 1e-2
#lr *= bs/48
lr /= 8
lr

0.00125

In [ ]:
# The Learner object we get is frozen by default, which means we only train the embeddings at first
learn.fit_one_cycle(1, lr, moms=(0.8,0.7), wd=0.1)


epoch,train_loss,valid_loss,accuracy,perplexity,time
0,7.855065,6.969272,0.110937,1063.448975,00:02


In [ ]:
learn.save('1epoch') # It will create a file in learn.path/models/ named "1epoch.pth" that can be used to tune later

In [ ]:
learn = learn.load('1epoch')

In [ ]:
# Then we unfreeze the model and fine-tune the whole thing for 10 epochs
learn.unfreeze()
learn.fit_one_cycle(8, lr/5, moms=(0.8,0.7), wd=0.1)


In [ ]:
# Once done we will save our encoder
learn.save_encoder('fwd_enc')


## Training the Backward Model

We can't directly train a bidirectional RNN for language modeling, but we can always ensemble a forward and backward model. fastai provides a pretrained forward and backawrd model, so we can repeat the previous step to fine-tune the pretrained backward model. The command language_model_learner checks the data object you pass to automatically decide if it should use the pretrained forward or backward model.

In [ ]:
learn = language_model_learner(data_bwd, AWD_LSTM, metrics=[accuracy, Perplexity()], path=path, wd=0.1).to_fp16(clip=0.1)

In [ ]:
# The rest of process is same

lr = 0.00125
learn.fit_one_cycle(1, lr, moms=(0.8,0.7), wd=0.1)

epoch,train_loss,valid_loss,accuracy,perplexity,time
0,8.320634,7.213185,0.081152,1357.208496,00:02


In [ ]:
learn.unfreeze()
learn.fit_one_cycle(8, lr/5, moms=(0.8,0.7), wd=0.1)


In [ ]:
learn.save_encoder('bwd_enc')


In [ ]:
# before tunning the classifier let us check how our model predicts some words given some context words from BBC news.

TEXT = "well change score playoff pool unfortunately time right certainly post new score rule tomorrow matter "
N_WORDS = 15
N_SENTENCES = 2
preds = [learn.predict(TEXT, N_WORDS, temperature=0.75) 
         for _ in range(N_SENTENCES)]

In [ ]:
preds

['well change score playoff pool unfortunately time right certainly post new score rule tomorrow matter  go game ds win time one player good need game point one play game two',
 'well change score playoff pool unfortunately time right certainly post new score rule tomorrow matter  xxbos xxbos thank thank get package mail e yes please thank contact require program use']

## Building a classifier

The classifier is a model that is a bit heavier, so we have lower the batch size.


In [ ]:
data_clas = (TextList.from_df(df, cols='Cleaned-text')
                   .split_by_rand_pct(0.1 , seed = 42)
                   .databunch(bs=bs, bptt=bptt))

#data_clas.save('data_clas.pkl')


/usr/local/lib/python3.7/dist-packages/fastai/core.py:302: VisibleDeprecationWarning: Creating an ndarray from ragged nested sequences (which is a list-or-tuple of lists-or-tuples-or ndarrays with different lengths or shapes) is deprecated. If you meant to do this, you must specify 'dtype=object' when creating the ndarray
  return np.array(a, dtype=dtype, **kwargs)


In [ ]:
del(data_clas)
data_clas = load_data(path, 'data_clas.pkl', bs=bs , bptt = bptt)
data_clas.show_batch()


idx,text
0,part last modify computer graphics resource list weekly post part last change april many faqs include list available archive site pit manager mit edu alias rtfm mit edu directory pub usenet news answer name faq archive appear archive name line
1,european sit carry add xxunk available radiance xxunk xxunk se pub sipp sipp tar z sipp scan line z buffer phong shade renderer jonas xxunk jonas xxunk se xxunk fr xxunk vm pray ray tracer spd nff many non spd
2,car radio park lot thankful strong radio miles pittsburgh xxunk usually good believe pick middle day xxbos anyone really believe cap beat let xxunk pen may xxunk one game put definitely lose one game remember regular season mean much come
3,might except might something x windows pcs know use think good email xxunk xxunk gov au dv x common xxunk xxunk corporation xxunk x software use dv x yet read xxunk xxunk send sound pretty great allow multiple do machine
4,g pts ashton berube macinnis musil nieuwendyk otto paslawski ranheim reichel roberts skrudland stern suter yawney san jose power play scorer g pts garpenlov gaudreau kisio odgers wood vancouver los angeles first period los angeles robitaille gretzky sandstrom vancouver babych


In [ ]:
# Like before we will load Data for Backward Model
data_clas_bwd = load_data(path, 'data_clas.pkl', bs=bs, bptt = bptt ,  backwards=True)
data_clas_bwd.show_batch()


idx,text
0,personally take hey streak unbeaten season regular far unbeaten game year last finals pen meet row blackhawks chicago tie streak playoff streak pen streak win game xxunk want might xxbos xxunk kiss xxunk hint siggraph xxunk xxunk g e surface
1,system software engineer navy xxunk sponsor maryland bethesda center research taylor david formerly center warfare surface naval division carderock june tuesday seminar reality virtual visualization scientific navy presentations call nd xxbos lssu maine xxbos xxunk also dve right xxunk wdve
2,use shit like play canucks xxunk first though big come probert win detroit think yes series playoff intense possibly war seven even maybe six wing though xxunk three two win might toronto playoffs lousy montreal find soon xxunk much leaf
3,basis xxunk state xxunk lease current oilers community business northlands office political hold power wake want pocklington xxbos source c even maybe algorithm understand easy anybody surface two intersections problem xxbos claim com ibm xxunk xxunk rate clock doubt seriously
4,idl lang comp also see available discount quantity educational cost progress version mac windows microsoft sun sgi rs ibm hp ultrix vms dec environments windows ms motif look open applications idl gui build system base widget map xxunk statistics basic


## Fine-tuning the forward classifier

The classifier needs a little less dropout, so we pass drop_mult=0.5 to multiply all the dropouts by this amount (it's easier than adjusting all the five different values manually). We don't load the pretrained model, but instead our fine-tuned encoder from the previous section.

In [ ]:
learn = text_classifier_learner(data_clas, AWD_LSTM, drop_mult=0.5, pretrained=False)
learn.load_encoder('fwd_enc')

AttributeError: ignored